In [42]:
import tensorflow as tf
from pprint import pprint

dataset_path = "/home/studio-lab-user/sign-language-image-detection/sign_datasets/sign-language-gesture-images-dataset/Gesture Image Data"

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    # Resize the images
    image_size=(96, 96),
    batch_size=64,
    shuffle=True
)

def normalize_image(image, label):
    precomputed_mean = [0.52732987, 0.4507709, 0.41209071]
    precomputed_std = [0.19798545, 0.23537221, 0.26049182]

    # Normalize the image
    image = (image / 255.0 - precomputed_mean) / precomputed_std
    return image, label

# Split the dataset into training and validation sets
# Define the split ratio
train_size = int(0.8 * len(dataset))  # 80% for training
val_size = len(dataset) - train_size   # 20% for validation

# Apply normalization to the datasets
train_dataset = train_dataset.map(normalize_image)
val_dataset = val_dataset.map(normalize_image)

# Create the training and validation datasets
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Prefetching for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)


Found 55500 files belonging to 37 classes.


In [43]:
from tqdm import tqdm

total = 0
unique_labels = set()
for batch_images, batch_labels in tqdm(train_dataset):
    total += len(batch_labels)
    for label in batch_labels:
        unique_labels.add(int(label))

print(total)
pprint(unique_labels)

100%|██████████| 694/694 [00:14<00:00, 47.17it/s]

44416
{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36}


In [44]:
from tqdm import tqdm

total = 0
unique_labels = set()
for batch_images, batch_labels in tqdm(val_dataset):
    total += len(batch_labels)
    for label in batch_labels:
        unique_labels.add(int(label))

print(total)
pprint(unique_labels)

100%|██████████| 174/174 [00:09<00:00, 18.70it/s]

11084
{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36}


In [45]:
model = tf.keras.Sequential(
    [
    tf.keras.layers.Flatten(input_shape=(96, 96, 3)),
    tf.keras.layers.Dense(500, activation="relu"),
    tf.keras.layers.Dense(len(class_names), activation="softmax"),
    ]
)

adam = tf.keras.optimizers.Adam(learning_rate=1e-5)

print(model.summary())

model.compile(
    optimizer=adam,
    loss="sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

model.fit(train_dataset, epochs=1, validation_data=val_dataset)

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_10 (Flatten)        (None, 27648)             0         
                                                                 
 dense_22 (Dense)            (None, 500)               13824500  
                                                                 
 dense_23 (Dense)            (None, 37)                18537     
                                                                 
Total params: 13843037 (52.81 MB)
Trainable params: 13843037 (52.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
694/694 [==============================] - 118s 169ms/step - loss: 11.5951 - accuracy: 0.7448 - val_loss: 1.3612 - val_accuracy: 0.9339


In [47]:
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPooling2D
from keras.models import Sequential, save_model

model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(96, 96 ,3) ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.20))
model.add(Dense(37, activation = 'softmax'))

model.summary()

model.compile(optimizer='Adam', metrics=['accuracy'], loss='sparse_categorical_crossentropy')
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=1)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 94, 94, 64)        1792      
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 47, 47, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_13 (Conv2D)          (None, 45, 45, 64)        36928     
                                                                 
 max_pooling2d_13 (MaxPooli  (None, 22, 22, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_14 (Conv2D)          (None, 20, 20, 64)        36928     
                                                                 
 max_pooling2d_14 (MaxPooli  (None, 10, 10, 64)      

In [49]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(96, 96, 3),
                                               include_top=False,
                                               weights='imagenet')

# We dont want to train this model
base_model.trainable = False

# Add custom layers on top of the base model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(class_names), activation='softmax')  # Adjust num_classes as needed
])

print(model.summary())

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=1)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Funct  (None, 3, 3, 1280)        2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d_3  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_30 (Dense)            (None, 128)               163968    
                                                                 
 dropout_8 (Dropout)         (None, 128)               0         
                                                                 
 dense_31 (Dense)            (None, 37)                4773      
                                                                 
Total params: 2426725 (9.26 MB)
Trainable params: 168